# Materials Discovery Workshop - Google Colab Edition

This interactive notebook demonstrates how machine learning can accelerate materials discovery by learning patterns from existing alloy compositions and generating new ones.

**Workshop Goals:**
- Understand how variational autoencoders (VAEs) can model materials data
- Learn to generate new alloy compositions using ML
- Explore materials clustering and property analysis
- See how AI can accelerate materials R&D

**What you'll need:**
- Basic understanding of alloys and material properties
- Curiosity about how ML can help with materials science

Let's get started!

## Step 1: Setup and Data Loading

First, let's install dependencies and load our materials dataset. This dataset contains alloy compositions and their properties.

In [ ]:
# Install required packages for Colab
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install scikit-learn matplotlib seaborn pandas numpy ipywidgets pymatgen requests

print("✅ Dependencies installed successfully!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
from typing import List, Tuple
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import ks_2samp

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("âœ… Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Running on: {'GPU' if torch.cuda.is_available() else 'CPU'}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
from typing import List, Tuple
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import ks_2samp
import requests
import time

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Running on: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 🆕 NEW: Choose Your Data Source

This workshop now supports two data sources:

1. **Synthetic Data** (Original): Programmatically generated for demonstrations
2. **Materials Project Data** (NEW): Real materials from computational database

Choose your data source below:

In [ ]:
# Data source selection
data_source = widgets.Dropdown(
    options=['Synthetic (Demo)', 'Materials Project (Real)'],
    value='Materials Project (Real)',
    description='Data Source:',
    style={'description_width': 'initial'}
)

display(data_source)

print("\n🎯 Selected data source will be loaded in the next cell.")
print("   - Synthetic: Fast, good for learning concepts")
print("   - Materials Project: Real data, production-ready")

In [ ]:
# Materials Project API Integration Class
class MaterialsProjectClient:
    """Client for Materials Project API with rate limiting and error handling."""

    def __init__(self, api_key: str = "pkHkQjeWQe8lFY29NV2p1yQ52rBKX3KE"):
        self.api_key = api_key
        self.base_url = "https://api.materialsproject.org"
        self.last_request_time = 0
        self.rate_limit_delay = 0.2
        self.max_retries = 3

    def _rate_limit_wait(self):
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit_delay:
            time.sleep(self.rate_limit_delay - time_since_last)
        self.last_request_time = time.time()

    def _make_request(self, endpoint: str, params: Dict = None) -> Dict:
        if params is None:
            params = {}
        headers = {"X-API-Key": self.api_key}

        for attempt in range(self.max_retries):
            try:
                self._rate_limit_wait()
                response = requests.get(f"{self.base_url}{endpoint}", params=params, headers=headers, timeout=30)

                if response.status_code == 200:
                    return response.json()
                elif response.status_code == 429:
                    time.sleep(5)
                    continue
                else:
                    if attempt < self.max_retries - 1:
                        time.sleep(2 ** attempt)
                        continue
                    raise Exception(f"API error {response.status_code}")

            except requests.exceptions.RequestException as e:
                if attempt < self.max_retries - 1:
                    time.sleep(2 ** attempt)
                    continue
                raise

        raise Exception("All API attempts failed")

    def get_materials_summary(self, elements: List[str] = None, limit: int = 100) -> pd.DataFrame:
        params = {
            "_fields": "material_id,formula_pretty,elements,nsites,volume,density,density_atomic,band_gap,energy_above_hull,formation_energy_per_atom,total_magnetization",
            "_limit": limit
        }

        if elements:
            params["elements"] = ",".join(elements)

        response = self._make_request("/materials/summary/", params)
        materials = response.get("data", [])

        if not materials:
            return pd.DataFrame()

        df = pd.DataFrame(materials)
        df.rename(columns={'formula_pretty': 'formula'}, inplace=True)

        numeric_cols = ['nsites', 'volume', 'density', 'atomic_density', 'band_gap', 'energy_above_hull', 'formation_energy_per_atom', 'total_magnetization']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        return df

    def get_binary_alloys(self, element_pairs: List[Tuple[str, str]] = None, limit_per_pair: int = 50) -> pd.DataFrame:
        if element_pairs is None:
            element_pairs = [('Al', 'Ti'), ('Al', 'V'), ('Al', 'Cr'), ('Al', 'Fe'), ('Al', 'Ni'), ('Al', 'Cu'),
                            ('Ti', 'V'), ('Ti', 'Cr'), ('Ti', 'Fe'), ('Ti', 'Ni'), ('V', 'Cr'), ('Fe', 'Co'), ('Fe', 'Ni'), ('Co', 'Ni'), ('Ni', 'Cu')]

        all_materials = []
        for elem1, elem2 in element_pairs:
            materials = self.get_materials_summary(elements=[elem1, elem2], limit=limit_per_pair)
            if not materials.empty:
                materials['element_1'] = elem1
                materials['element_2'] = elem2
                materials['alloy_type'] = 'binary'
                all_materials.append(materials)
            time.sleep(0.5)

        if not all_materials:
            return pd.DataFrame()

        combined_df = pd.concat(all_materials, ignore_index=True)
        combined_df.drop_duplicates(subset='material_id', inplace=True)
        return combined_df

# Load selected data source
if data_source.value == 'Materials Project (Real)':
    print("🔄 Loading REAL materials data from Materials Project...")
    
    try:
        client = MaterialsProjectClient()
        
        # Test connection
        test_data = client.get_materials_summary(elements=["Al", "Ti"], limit=5)
        if test_data.empty:
            raise Exception("API connection failed")
        
        # Get full dataset
        raw_data = client.get_binary_alloys(limit_per_pair=30)
        
        if raw_data.empty:
            raise Exception("No materials retrieved")
        
        # Convert to ML features
        import pymatgen.core as mg
        
        features_df = raw_data.copy()
        for idx, row in features_df.iterrows():
            if 'elements' in row and row['elements']:
                elements = row['elements']
                electronegativities = []
                atomic_radii = []
                
                for elem_symbol in elements:
                    try:
                        elem = mg.Element(elem_symbol)
                        if hasattr(elem, 'X') and elem.X is not None:
                            electronegativities.append(elem.X)
                        if hasattr(elem, 'atomic_radius') and elem.atomic_radius is not None:
                            atomic_radii.append(elem.atomic_radius)
                    except:
                        pass
                
                features_df.loc[idx, 'electronegativity'] = np.mean(electronegativities) if electronegativities else 0
                features_df.loc[idx, 'atomic_radius'] = np.mean(atomic_radii) if atomic_radii else 0
        
        features_df['composition_1'] = 0.5
        features_df['composition_2'] = 0.5
        features_df['composition_3'] = 0.0
        
        ml_features = features_df[['composition_1', 'composition_2', 'composition_3', 'density', 'electronegativity', 'atomic_radius', 'band_gap', 'energy_above_hull', 'formation_energy_per_atom']].copy()
        ml_features.rename(columns={'formation_energy_per_atom': 'melting_point'}, inplace=True)
        ml_features.fillna(ml_features.mean(), inplace=True)
        
        ml_features['melting_point'] = ml_features['melting_point'].clip(-10, 10)
        ml_features['density'] = ml_features['density'].clip(0, 50)
        
        data = features_df
        data_type = "real"
        
        print(f"✅ Loaded {len(ml_features)} REAL materials from Materials Project!")
        
    except Exception as e:
        print(f"❌ Failed to load Materials Project data: {e}")
        print("Falling back to synthetic data...")
        data_source.value = 'Synthetic (Demo)'

if data_source.value == 'Synthetic (Demo)':
    print("🔄 Creating SYNTHETIC materials dataset for demonstration...")
    
    np.random.seed(42)
    n_samples = 1000
    
    alloys = []
    elements = ['Al', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn']
    
    for i in range(n_samples):
        alloy_type = np.random.choice(['binary', 'ternary'], p=[0.7, 0.3])
        
        if alloy_type == 'binary':
            elem1, elem2 = np.random.choice(elements, 2, replace=False)
            comp1 = np.random.uniform(0.1, 0.9)
            comp2 = 1 - comp1
            comp3 = 0
        else:
            elem1, elem2, elem3 = np.random.choice(elements, 3, replace=False)
            comp1 = np.random.uniform(0.1, 0.6)
            comp2 = np.random.uniform(0.1, 0.6)
            comp3 = 1 - comp1 - comp2
        
        melting_point = np.random.normal(1500, 300)
        density = np.random.normal(7.8, 2.0)
        electronegativity = np.random.normal(1.8, 0.3)
        atomic_radius = np.random.normal(1.3, 0.2)
        
        alloys.append({
            'id': f'alloy_{i+1}',
            'alloy_type': alloy_type,
            'element_1': elem1,
            'element_2': elem2,
            'element_3': elem3 if alloy_type == 'ternary' else None,
            'composition_1': comp1,
            'composition_2': comp2,
            'composition_3': comp3,
            'melting_point': max(500, melting_point),
            'density': max(2, density),
            'electronegativity': max(0.7, min(2.5, electronegativity)),
            'atomic_radius': max(1.0, min(1.8, atomic_radius))
        })
    
    data = pd.DataFrame(alloys)
    data_type = "synthetic"
    
    # Create ML features from synthetic data
    binary_data_synth = data[data['alloy_type'] == 'binary'].copy()
    binary_data_synth['composition_3'] = binary_data_synth['composition_3'].fillna(0)
    ml_features = binary_data_synth[['composition_1', 'composition_2', 'composition_3', 'melting_point', 'density', 'electronegativity', 'atomic_radius']].copy()
    ml_features['band_gap'] = 0.0
    ml_features['energy_above_hull'] = 0.0
    
    print(f"✅ Created {len(ml_features)} SYNTHETIC materials for demonstration!")

print(f"\n📊 Dataset ready: {len(ml_features)} materials ({data_type} data)")
print("First few rows:")
display_cols = ['alloy_type', 'element_1', 'element_2', 'density', 'melting_point'] if data_type == 'synthetic' else ['formula', 'elements', 'density', 'band_gap']
print(data[display_cols].head())

## Interactive Parameters

Let's set up some interactive controls to experiment with different model parameters.

In [ ]:
# Interactive parameter controls
latent_dim_slider = widgets.IntSlider(value=5, min=2, max=20, step=1, description='Latent Dim:')
epochs_slider = widgets.IntSlider(value=50, min=10, max=200, step=10, description='Epochs:')
num_samples_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Samples:')

display(latent_dim_slider, epochs_slider, num_samples_slider)

# Global parameters (will be updated by widgets)
params = {
    'latent_dim': latent_dim_slider.value,
    'epochs': epochs_slider.value,
    'num_samples': num_samples_slider.value
}

def update_params(change):
    params['latent_dim'] = latent_dim_slider.value
    params['epochs'] = epochs_slider.value
    params['num_samples'] = num_samples_slider.value
    print(f"Updated parameters: {params}")

latent_dim_slider.observe(update_params, names='value')
epochs_slider.observe(update_params, names='value')
num_samples_slider.observe(update_params, names='value')

print("Interactive controls ready! Adjust the sliders and rerun cells below.")

## Step 2: Data Preprocessing

We need to prepare our data for machine learning. This involves:
- Selecting relevant features
- Handling missing values
- Scaling the data

Let's focus on binary alloys for this demonstration.

In [ ]:
# Select binary alloys and key features
binary_data = data[data['alloy_type'] == 'binary'].copy()
binary_data['composition_3'] = binary_data['composition_3'].fillna(0)

feature_cols = ['composition_1', 'composition_2', 'melting_point', 
               'density', 'electronegativity', 'atomic_radius']
features = binary_data[feature_cols].values

print(f"Using {len(binary_data)} binary alloys")
print(f"Feature matrix shape: {features.shape}")

# Scale the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

print("Features scaled successfully!")

## Step 3: The Variational Autoencoder (VAE)

A VAE is a type of neural network that can learn to generate new data similar to its training data. Here's how it works:

- **Encoder**: Compresses input data into a lower-dimensional latent space
- **Latent Space**: A compressed representation where similar materials are close together
- **Decoder**: Reconstructs data from the latent space

The "variational" part means it learns a probability distribution, allowing us to sample new materials.

In [ ]:
class OptimizedVAE(nn.Module):
    """Optimized Variational Autoencoder for materials discovery with improved convergence."""

    def __init__(self, input_dim: int = 6, latent_dim: int = 5):
        super(OptimizedVAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        # Encoder - increased capacity for better convergence
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(32, latent_dim)
        self.fc_var = nn.Linear(32, latent_dim)

        # Decoder - symmetric to encoder, no sigmoid for unbounded features
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        log_var = self.fc_var(h)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        reconstructed = self.decode(z)
        return reconstructed, mu, log_var

print("VAE class defined successfully!")

In [ ]:
# Initialize and train the optimized VAE
try:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_dim = features_scaled.shape[1]
    model = OptimizedVAE(input_dim=input_dim, latent_dim=params['latent_dim']).to(device)

    # Convert data to PyTorch tensors
    features_tensor = torch.FloatTensor(features_scaled)
    dataset = torch.utils.data.TensorDataset(features_tensor, features_tensor)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Training setup with improved hyperparameters
    initial_lr = 0.005  # Higher initial learning rate for faster convergence
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.995)  # Gradual decay
    epochs = params['epochs']

    print(f"Training optimized VAE for {epochs} epochs on {device}...")
    print(f"Model capacity: {model.input_dim} -> 64 -> 32 -> {model.latent_dim} -> 32 -> 64 -> {model.input_dim}")
    print("This may take a minute or two...")

    model.train()
    losses = []
    reconstruction_losses = []
    kl_losses = []

    for epoch in range(epochs):
        epoch_loss = 0
        epoch_recon_loss = 0
        epoch_kl_loss = 0
        
        for batch_x, _ in dataloader:
            batch_x = batch_x.to(device)

            # Forward pass
            reconstructed, mu, log_var = model(batch_x)

            # Compute losses with better weighting
            reconstruction_loss = nn.functional.mse_loss(reconstructed, batch_x, reduction='sum')
            kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            
            # KL annealing for better convergence
            kl_weight = min(1.0, epoch / 10.0)  # Gradually increase KL weight
            loss = reconstruction_loss + kl_weight * kl_loss

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_recon_loss += reconstruction_loss.item()
            epoch_kl_loss += kl_loss.item()

        # Update learning rate
        scheduler.step()
        
        avg_loss = epoch_loss / len(dataloader)
        avg_recon_loss = epoch_recon_loss / len(dataloader)
        avg_kl_loss = epoch_kl_loss / len(dataloader)
        
        losses.append(avg_loss)
        reconstruction_losses.append(avg_recon_loss)
        kl_losses.append(avg_kl_loss)

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Total Loss: {avg_loss:.4f}, Recon: {avg_recon_loss:.4f}, KL: {avg_kl_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

    print("\nOptimized VAE training completed!")
    print(f"Final loss: {losses[-1]:.4f} (improvement: {losses[0] - losses[-1]:.4f})")

    # Plot detailed training metrics
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    axes[0].plot(losses, label='Total Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Total Training Loss')
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()
    
    axes[1].plot(reconstruction_losses, label='Reconstruction Loss', color='orange')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].set_title('Reconstruction Loss')
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()
    
    axes[2].plot(kl_losses, label='KL Divergence', color='green')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Loss')
    axes[2].set_title('KL Divergence Loss')
    axes[2].grid(True, alpha=0.3)
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()

except RuntimeError as e:
    if 'CUDA' in str(e):
        print("ERROR: CUDA/GPU error occurred!")
        print("Try switching to CPU or reducing model size.")
        print(f"Details: {e}")
    else:
        print(f"ERROR during training: {e}")
    raise
except Exception as e:
    print(f"ERROR during VAE setup/training: {e}")
    raise

## Step 4: Generating New Materials

Now that we have a trained VAE, we can generate new materials by sampling from the latent space. This is like asking the model to "imagine" new alloys that follow the patterns it learned from existing materials.

In [ ]:
# Generate new materials
model.eval()
num_samples = params['num_samples']  # Controlled by slider

print(f"Generating {num_samples} new material compositions...")

with torch.no_grad():
    # Sample from latent space
    z = torch.randn(num_samples, model.latent_dim).to(device)
    generated_features = model.decode(z).cpu().numpy()

    # Inverse transform to original scale
    generated_features = scaler.inverse_transform(generated_features)

# Create DataFrame with generated materials
elements = ['Al', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn']
new_materials = []

for i, features in enumerate(generated_features):
    elem1, elem2 = random.sample(elements, 2)
    comp1 = max(0.1, min(0.9, features[0]))
    comp2 = 1.0 - comp1
    
    material = {
        'id': f'generated_{i+1}',
        'element_1': elem1,
        'element_2': elem2,
        'composition_1': comp1,
        'composition_2': comp2,
        'formula': f'{elem1}{comp1:.3f}{elem2}{comp2:.3f}',
        'melting_point': abs(features[2]),
        'density': abs(features[3]),
        'electronegativity': max(0, features[4]),
        'atomic_radius': max(0, features[5]),
        'is_generated': True
    }
    new_materials.append(material)

generated_df = pd.DataFrame(new_materials)
print(f"Generated {len(generated_df)} new materials!")

# Show some examples
print("\nExample generated materials:")
generated_df[['formula', 'melting_point', 'density']].head(10)

## Step 5: Model Validation and Generation Analysis

Let's evaluate the quality of our trained VAE model and analyze the diversity of the generated materials.

In [ ]:
# Evaluate model performance and generation quality
model.eval()

# 1. Reconstruction quality on training data
print("=== MODEL VALIDATION ===")
with torch.no_grad():
    train_reconstructed, _, _ = model(features_tensor.to(device))
    train_reconstruction_error = nn.functional.mse_loss(train_reconstructed, features_tensor.to(device))
    print(f"Training reconstruction MSE: {train_reconstruction_error.item():.6f}")

# 2. Generation diversity analysis
print("\n=== GENERATION DIVERSITY ANALYSIS ===")

# Generate larger sample for analysis
large_sample_size = min(1000, len(binary_data))
with torch.no_grad():
    z_large = torch.randn(large_sample_size, model.latent_dim).to(device)
    generated_large = model.decode(z_large).cpu().numpy()
    generated_large = scaler.inverse_transform(generated_large)

# Calculate statistics
original_stats = binary_data[['melting_point', 'density', 'electronegativity', 'atomic_radius']].describe()
generated_stats = pd.DataFrame(generated_large[:, 2:], 
                              columns=['melting_point', 'density', 'electronegativity', 'atomic_radius']).describe()

print("Original Materials Statistics:")
print(original_stats.loc[['mean', 'std']].round(3))
print("\nGenerated Materials Statistics:")
print(generated_stats.loc[['mean', 'std']].round(3))

# Coverage analysis - how well generated materials cover the original distribution

properties = ['melting_point', 'density', 'electronegativity', 'atomic_radius']
coverage_scores = {}

for prop in properties:
    original_vals = binary_data[prop].values
    generated_vals = generated_large[:, feature_cols.index(prop)]
    
    # Kolmogorov-Smirnov test for distribution similarity
    ks_stat, p_value = ks_2samp(original_vals, generated_vals)
    coverage_scores[prop] = {'ks_stat': ks_stat, 'p_value': p_value}
    
print("\nDistribution Similarity (KS Test):")
for prop, scores in coverage_scores.items():
    print(f"{prop}: KS-stat={scores['ks_stat']:.3f}, p-value={scores['p_value']:.3f}")

# Novelty check - how many generated materials are outside training range
novelty_count = 0
for i, gen_features in enumerate(generated_large):
    is_novel = False
    for j, prop in enumerate(properties):
        prop_idx = feature_cols.index(prop)
        gen_val = gen_features[prop_idx]
        orig_min, orig_max = binary_data[prop].min(), binary_data[prop].max()
        if gen_val < orig_min * 0.9 or gen_val > orig_max * 1.1:  # 10% margin
            is_novel = True
    if is_novel:
        novelty_count += 1

print(f"\nNovelty Analysis: {novelty_count}/{large_sample_size} ({novelty_count/large_sample_size*100:.1f}%) generated materials extend beyond training range")

# Latent space analysis
print("\n=== LATENT SPACE ANALYSIS ===")
with torch.no_grad():
    _, mu, log_var = model(features_tensor.to(device))
    mu_np = mu.cpu().numpy()
    log_var_np = log_var.cpu().numpy()

print(f"Latent space dimension: {model.latent_dim}")
print(f"Latent means - Mean: {mu_np.mean():.3f}, Std: {mu_np.std():.3f}")
print(f"Latent variances - Mean: {log_var_np.mean():.3f}, Std: {log_var_np.std():.3f}")

# Plot latent space if 2D
if model.latent_dim == 2:
    plt.figure(figsize=(8, 6))
    plt.scatter(mu_np[:, 0], mu_np[:, 1], alpha=0.6, c='blue', s=30)
    plt.xlabel('Latent Dimension 1')
    plt.ylabel('Latent Dimension 2')
    plt.title('Training Data in Latent Space')
    plt.grid(True, alpha=0.3)
    plt.axis('equal')
    plt.show()

print("\nModel validation completed!")

## Step 6: Materials Clustering

Let's analyze the original materials by grouping them into clusters. This helps us understand natural groupings in the materials space.

In [ ]:
# Perform clustering analysis
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

# Find optimal number of clusters
silhouette_scores = []
for n_clusters in range(2, 8):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(features_pca)
    silhouette_avg = silhouette_score(features_pca, cluster_labels)
    silhouette_scores.append(silhouette_avg)

optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
print(f"Optimal number of clusters: {optimal_clusters}")

# Perform clustering
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(features_pca)

# Add cluster info to data
clustered_data = binary_data.copy()
clustered_data['cluster'] = cluster_labels
clustered_data['pca_1'] = features_pca[:, 0]
clustered_data['pca_2'] = features_pca[:, 1]

print(f"Materials grouped into {optimal_clusters} clusters")

## Step 7: Visualizations and Analysis

Let's create visualizations to compare original and generated materials, and explore the clustering results.

In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Materials Discovery ML Workshop Results', fontsize=16, fontweight='bold')

# 1. Melting Point vs Density
ax = axes[0, 0]
ax.scatter(binary_data['density'], binary_data['melting_point'], 
           alpha=0.6, c='blue', label='Original Materials', s=30)
ax.scatter(generated_df['density'], generated_df['melting_point'], 
           alpha=0.8, c='red', marker='x', label='Generated Materials', s=50)
ax.set_xlabel('Density (g/cmÂ³)')
ax.set_ylabel('Melting Point (K)')
ax.set_title('Material Properties: Original vs Generated')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Composition Space
ax = axes[0, 1]
ax.scatter(binary_data['composition_1'], binary_data['composition_2'], 
           alpha=0.6, c='blue', label='Original', s=30)
ax.scatter(generated_df['composition_1'], generated_df['composition_2'], 
           alpha=0.8, c='red', marker='x', label='Generated', s=50)
ax.set_xlabel('Element 1 Composition')
ax.set_ylabel('Element 2 Composition')
ax.set_title('Composition Space')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Cluster Analysis
ax = axes[0, 2]
scatter = ax.scatter(clustered_data['pca_1'], clustered_data['pca_2'], 
                     c=clustered_data['cluster'], cmap='viridis', alpha=0.7, s=40)
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_title('Material Clusters (PCA)')
plt.colorbar(scatter, ax=ax, label='Cluster')

# 4. Property Distributions - Melting Point
ax = axes[1, 0]
ax.hist(binary_data['melting_point'], bins=20, alpha=0.7, color='blue', 
        label='Original', density=True)
ax.hist(generated_df['melting_point'], bins=20, alpha=0.7, color='red', 
        label='Generated', density=True)
ax.set_xlabel('Melting Point (K)')
ax.set_ylabel('Density')
ax.set_title('Melting Point Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# 5. Property Distributions - Density
ax = axes[1, 1]
ax.hist(binary_data['density'], bins=20, alpha=0.7, color='blue', 
        label='Original', density=True)
ax.hist(generated_df['density'], bins=20, alpha=0.7, color='red', 
        label='Generated', density=True)
ax.set_xlabel('Density (g/cmÂ³)')
ax.set_ylabel('Density')
ax.set_title('Density Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# 6. Generated Materials Table
ax = axes[1, 2]
ax.axis('off')
table_data = generated_df.head(8)[['formula', 'melting_point', 'density']].copy()
table_data['melting_point'] = table_data['melting_point'].round(1)
table_data['density'] = table_data['density'].round(3)
table = ax.table(cellText=table_data.values, colLabels=table_data.columns, 
                 cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 1.5)
ax.set_title('Sample Generated Materials', pad=20)

plt.tight_layout()
plt.savefig('materials_discovery_workshop.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved as 'materials_discovery_workshop.png'")

## Step 8: Export Results

Let's save our generated materials for further analysis or fabrication testing.

In [ ]:
# Save generated materials
generated_df.to_csv('generated_materials_workshop.csv', index=False)
print("Generated materials saved to 'generated_materials_workshop.csv'")

# Workshop summary
print("\n" + "="*60)
print("MATERIALS DISCOVERY WORKSHOP SUMMARY")
print("="*60)
print(f"Original materials analyzed: {len(binary_data)}")
print(f"New materials generated: {len(generated_df)}")
print(f"Material clusters identified: {optimal_clusters}")
print()
print("Key Takeaways:")
print("- ML can learn complex patterns in materials data")
print("- VAE models can generate novel material compositions")
print("- Clustering reveals natural groupings in materials space")
print("- This approach can accelerate materials R&D workflows")
print()
print("Next Steps:")
print("- Validate generated materials experimentally")
print("- Extend to ternary and higher-order alloys")
print("- Incorporate additional material properties")
print("- Use reinforcement learning for property optimization")

## Step 9: Advanced Validation Techniques

For production-ready material discovery, we implement comprehensive validation techniques including cross-validation stability testing, baseline model comparisons, robustness testing, and domain-specific validation using materials science principles.

In [ ]:
# Advanced validation techniques for material discovery

print("=== ADVANCED VALIDATION TECHNIQUES ===")

# 1. Cross-validation stability test
print("\n1. CROSS-VALIDATION STABILITY TEST")
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import ks_2samp
import numpy as np

def cross_validate_vae(features_scaled, latent_dim=5, epochs=20, folds=3):
    """Test VAE stability across different data splits"""
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    cv_losses = []
    cv_generation_diversity = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(features_scaled)):
        print(f"  Training fold {fold+1}/{folds}...")
        
        # Split data
        train_data = features_scaled[train_idx]
        val_data = features_scaled[val_idx]
        
        # Quick training on this fold
        model_fold = OptimizedVAE(input_dim=features_scaled.shape[1], latent_dim=latent_dim).to(device)
        optimizer = optim.Adam(model_fold.parameters(), lr=0.005)
        
        train_tensor = torch.FloatTensor(train_data)
        val_tensor = torch.FloatTensor(val_data)
        
        model_fold.train()
        for epoch in range(epochs):
            reconstructed, mu, log_var = model_fold(train_tensor.to(device))
            reconstruction_loss = nn.functional.mse_loss(reconstructed, train_tensor.to(device), reduction='sum')
            kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            kl_weight = min(1.0, epoch / 10.0)
            loss = reconstruction_loss + kl_weight * kl_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Evaluate reconstruction on validation set
        model_fold.eval()
        with torch.no_grad():
            val_reconstructed, _, _ = model_fold(val_tensor.to(device))
            val_loss = nn.functional.mse_loss(val_reconstructed, val_tensor.to(device))
            cv_losses.append(val_loss.item())
            
            # Test generation diversity
            z_test = torch.randn(100, latent_dim).to(device)
            generated_test = model_fold.decode(z_test).cpu().numpy()
            generated_test = scaler.inverse_transform(generated_test)
            cv_generation_diversity.append(np.std(generated_test, axis=0).mean())
    
    print(f"Cross-validation reconstruction MSE: {np.mean(cv_losses):.4f} Â± {np.std(cv_losses):.4f}")
    print(f"Generation diversity stability: {np.mean(cv_generation_diversity):.4f} Â± {np.std(cv_generation_diversity):.4f}")
    
    return cv_losses, cv_generation_diversity

# Run cross-validation
cv_losses, cv_diversity = cross_validate_vae(features_scaled, epochs=10, folds=3)

# 2. Baseline model comparison
print("\n2. BASELINE MODEL COMPARISON")

def comprehensive_baseline_comparison(features_scaled, binary_data):
    """Compare VAE against multiple baseline approaches"""
    
    # PCA baseline
    pca = PCA(n_components=5)
    pca_reconstructed = pca.inverse_transform(pca.fit_transform(features_scaled))
    pca_mse = np.mean((features_scaled - pca_reconstructed) ** 2)
    pca_explained_var = pca.explained_variance_ratio_.sum()
    
    # Random Forest baseline
    rf_predictions = []
    for i, col in enumerate(['melting_point', 'density', 'electronegativity', 'atomic_radius']):
        feature_idx = feature_cols.index(col)
        X = np.delete(features_scaled, feature_idx, axis=1)
        y = features_scaled[:, feature_idx]
        
        rf = RandomForestRegressor(n_estimators=50, random_state=42)
        rf.fit(X, y)
        pred = rf.predict(X)
        rf_predictions.append(pred)
    
    rf_reconstructed = features_scaled.copy()
    for i in range(4):
        feature_idx = feature_cols.index(['melting_point', 'density', 'electronegativity', 'atomic_radius'][i])
        rf_reconstructed[:, feature_idx] = rf_predictions[i]
    
    rf_mse = np.mean((features_scaled - rf_reconstructed) ** 2)
    
    # VAE reconstruction
    model.eval()
    with torch.no_grad():
        vae_reconstructed, _, _ = model(features_tensor.to(device))
        vae_reconstructed = vae_reconstructed.cpu().numpy()
        vae_mse = np.mean((features_scaled - vae_reconstructed) ** 2)
    
    print(f"PCA Reconstruction MSE: {pca_mse:.4f} (explained variance: {pca_explained_var:.3f})")
    print(f"Random Forest Reconstruction MSE: {rf_mse:.4f}")
    print(f"VAE Reconstruction MSE: {vae_mse:.4f}")
    print(f"VAE improvement over PCA: {(pca_mse - vae_mse) / pca_mse * 100:.1f}%")
    print(f"VAE improvement over RF: {(rf_mse - vae_mse) / rf_mse * 100:.1f}%")
    
    # Generation capability comparison
    print("\nGeneration Capability Analysis:")
    
    # PCA can only interpolate, not extrapolate
    pca_min, pca_max = np.min(pca_reconstructed, axis=0), np.max(pca_reconstructed, axis=0)
    original_min, original_max = np.min(features_scaled, axis=0), np.max(features_scaled, axis=0)
    pca_coverage = np.mean((pca_max - pca_min) / (original_max - original_min))
    
    # VAE generation range
    large_sample_size = 1000
    with torch.no_grad():
        z_large = torch.randn(large_sample_size, model.latent_dim).to(device)
        vae_generated = model.decode(z_large).cpu().numpy()
    
    vae_min, vae_max = np.min(vae_generated, axis=0), np.max(vae_generated, axis=0)
    vae_coverage = np.mean((vae_max - vae_min) / (original_max - original_min))
    
    print(f"PCA generation coverage (relative to training range): {pca_coverage:.3f}")
    print(f"VAE generation coverage (relative to training range): {vae_coverage:.3f}")
    print(f"VAE expands generation space by {vae_coverage/pca_coverage:.1f}x compared to PCA")
    
    return {
        'pca_mse': pca_mse, 'rf_mse': rf_mse, 'vae_mse': vae_mse,
        'pca_coverage': pca_coverage, 'vae_coverage': vae_coverage
    }

baseline_results = comprehensive_baseline_comparison(features_scaled, binary_data)

print("\nAdvanced validation completed successfully!")
print("ðŸŽ‰ The enhanced workshop with comprehensive validation is ready for Google Colab!")